
### Requisitos para o arquivo estadios.csv

Objetivo: realizar processo de ETL e disponibilizar os dados em camada adequada.

1. Ler o arquivo csv do Datalake [ok]
2. Renomear colunas para português usando snake case [ok]
3. Definir a tipagem de colunas correta [ok]
4. Coluna com data de cosumo do arquivo [ok]
5. Salvar dados em formato parquet na camada adequada [ok]

In [1]:
%run ../../Configuração-e-Utilitários/Variaveis.ipynb

In [2]:
%run ../../Configuração-e-Utilitários/Funções.ipynb


### Lendo os dados do arquivo

In [5]:
estadios_df = (spark.read
               .option("encoding", "UTF-16")
               .option("header", "true") # Detecção do cabeçalho
               .option("inferSchema", "true") # Detecção do tipo de dados
               .csv(f"{caminho_bronze}/estadios.csv") # Caminho do arquivo
)
estadios_df.show()

+-----+--------------------+--------------------+--------------------+-------+--------+-------+--------------------+
|   id|                name|             address|                city|country|capacity|surface|               image|
+-----+--------------------+--------------------+--------------------+-------+--------+-------+--------------------+
|  204|Estadio Jornalist...|Rua Professor Eur...|Rio de Janeiro, R...| Brazil|   78838|  grass|https://media-2.a...|
|  216|    Arena Fonte Nova|Rua Lions Club, N...|     Salvador, Bahia| Brazil|   56500|  grass|https://media-1.a...|
|  244|Estádio José Pinh...|Avenida Padre Cac...|Porto Alegre, Rio...| Brazil|   50128|  grass|https://media-3.a...|
|  218|Estádio Nilton Sa...|Rua Arquias Corde...|      Rio de Janeiro| Brazil|   46931|  grass|https://media-1.a...|
|  258|      Allianz Parque|Rua Turiaçu 1840,...|São Paulo, São Paulo| Brazil|   43713|  grass|https://media-2.a...|
|  259|Estádio Durival d...|Rua Engenheiro Re...|    Curitiba, P

In [6]:
estadios_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- capacity: integer (nullable = true)
 |-- surface: string (nullable = true)
 |-- image: string (nullable = true)



### Renomeando as colunas

In [7]:
estadios_df = estadios_df \
  .withColumnRenamed("id", "id_estadio") \
  .withColumnRenamed("name", "nm_estadio") \
  .withColumnRenamed("address", "endereço") \
  .withColumnRenamed("city", "cidade") \
  .withColumnRenamed("country", "pais") \
  .withColumnRenamed("capacity", "capacidade") \
  .withColumnRenamed("surface", "superficie") \
  .withColumnRenamed("image", "link_imagem") \

estadios_df.show()


+----------+--------------------+--------------------+--------------------+------+----------+----------+--------------------+
|id_estadio|          nm_estadio|            endereço|              cidade|  pais|capacidade|superficie|         link_imagem|
+----------+--------------------+--------------------+--------------------+------+----------+----------+--------------------+
|       204|Estadio Jornalist...|Rua Professor Eur...|Rio de Janeiro, R...|Brazil|     78838|     grass|https://media-2.a...|
|       216|    Arena Fonte Nova|Rua Lions Club, N...|     Salvador, Bahia|Brazil|     56500|     grass|https://media-1.a...|
|       244|Estádio José Pinh...|Avenida Padre Cac...|Porto Alegre, Rio...|Brazil|     50128|     grass|https://media-3.a...|
|       218|Estádio Nilton Sa...|Rua Arquias Corde...|      Rio de Janeiro|Brazil|     46931|     grass|https://media-1.a...|
|       258|      Allianz Parque|Rua Turiaçu 1840,...|São Paulo, São Paulo|Brazil|     43713|     grass|https://media-

### Criando coluna de data de consumo

In [8]:
# Função para adição de data de ingestão
estadios_df = adicionar_data_de_ingestao(
    df = estadios_df,
    nome_coluna = 'data_de_ingestao',
    incluir_hora = False,
    add_tempo = -3,
    unidade_de_medida = 'horas'
)

estadios_df.show()

+----------+--------------------+--------------------+--------------------+------+----------+----------+--------------------+----------------+
|id_estadio|          nm_estadio|            endereço|              cidade|  pais|capacidade|superficie|         link_imagem|data_de_ingestao|
+----------+--------------------+--------------------+--------------------+------+----------+----------+--------------------+----------------+
|       204|Estadio Jornalist...|Rua Professor Eur...|Rio de Janeiro, R...|Brazil|     78838|     grass|https://media-2.a...|      2024-05-04|
|       216|    Arena Fonte Nova|Rua Lions Club, N...|     Salvador, Bahia|Brazil|     56500|     grass|https://media-1.a...|      2024-05-04|
|       244|Estádio José Pinh...|Avenida Padre Cac...|Porto Alegre, Rio...|Brazil|     50128|     grass|https://media-3.a...|      2024-05-04|
|       218|Estádio Nilton Sa...|Rua Arquias Corde...|      Rio de Janeiro|Brazil|     46931|     grass|https://media-1.a...|      2024-05-04|

In [9]:
estadios_df.printSchema()

root
 |-- id_estadio: string (nullable = true)
 |-- nm_estadio: string (nullable = true)
 |-- endereço: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- capacidade: integer (nullable = true)
 |-- superficie: string (nullable = true)
 |-- link_imagem: string (nullable = true)
 |-- data_de_ingestao: date (nullable = true)



### Salvando em formato parquet

In [12]:
estadios_df.write.mode('overwrite').parquet(f"{caminho_silver}/estadios")